In [1]:
import os
path = '/Users/danmontesi/Desktop/recsys2019'
os.chdir(path)
print(os.getcwd())

/Users/danmontesi/Desktop/recsys2019


In [2]:
import data
import pandas as pd
import numpy as np

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [4]:
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

In [5]:
def existsIndex(df, idx):
    try:
        df.loc[idx]
        return True
    except KeyError:
        return False

In [6]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except TypeError:
        return False
    except ValueError:
        return False

In [7]:
def getNumOfAction(df, action_type):
    return len(df[df["action_type"] == action_type])

In [8]:
dataset = data.train_df("full")


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
df = dataset.copy()

In [50]:
df[df["session_id"] == "c0cdade1ed014"]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
620,0X3OMZZH37ZS,c0cdade1ed014,1541085782,1,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
621,0X3OMZZH37ZS,c0cdade1ed014,1541085782,2,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
622,0X3OMZZH37ZS,c0cdade1ed014,1541085809,3,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
623,0X3OMZZH37ZS,c0cdade1ed014,1541085809,4,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
624,0X3OMZZH37ZS,c0cdade1ed014,1541085809,5,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
625,0X3OMZZH37ZS,c0cdade1ed014,1541085809,6,interaction item image,434856,NL,"Souillac, France",desktop,NaN,NaN,NaN
626,0X3OMZZH37ZS,c0cdade1ed014,1541085821,7,clickout item,1358429,NL,"Souillac, France",desktop,NaN,1556793|434856|3206344|49682|1358429|658821|10...,64|83|65|82|63|71|96|105|60|87|66|62|85|65|97|...
627,0X3OMZZH37ZS,c0cdade1ed014,1541085912,8,clickout item,434856,NL,"Souillac, France",desktop,NaN,1556793|434856|3206344|49682|1358429|658821|10...,64|83|65|82|63|71|96|105|60|87|66|62|85|65|97|...


In [480]:
def func(x):
    
    y = x[x['action_type'] == 'clickout item']
    
    #keep only sessions until last clickout

    clk = y.tail(1)
    
    if existsIndex(x, clk.index - 1):
        #exists numeric reference before
        if RepresentsInt(x.loc[clk.index - 1]['reference']):
            
            clicked = x.loc[clk.index]['reference'].values[0]
            
            x = x[ x["step"] < int(clk["step"])]
            
            impressions = list(map(int, clk['impressions'].values[0].split('|')))
    
            
            
            x_only_numeric_ref = x[pd.to_numeric(x['reference'], errors='coerce').notnull()]
            
            interacted = list(x_only_numeric_ref.reference)
            
            # feature computation
            info_perc = getNumOfAction(x, "interaction item info") / len(x_only_numeric_ref)
            
            rating_perc = getNumOfAction(x, "interaction item rating") / len(x_only_numeric_ref)
            
            deals_perc = getNumOfAction(x, "interaction item deals") / len(x_only_numeric_ref)
            
            image_perc = getNumOfAction(x, "interaction item image") / len(x_only_numeric_ref)
            
            clickout_count = getNumOfAction(x, "clickout item")
            
            prev_clickout = 0
            if clicked in list(y[:-1]["reference"]):
                prev_clickout = 1
            
            poi_count = getNumOfAction(x, "search for poi")
            
            search_destination_count = getNumOfAction(x, "search for destination")
            
            search_item_count = getNumOfAction(x, "search for item")
            
            filter_sel_count = getNumOfAction(x, "filter selection")
            
            sort_change_count = getNumOfAction(x, "change of sort order")
            
            last_time_from_first_interaction = int(clk['timestamp']) - int(list(x_only_numeric_ref['timestamp'])[0])

            # time elapsed
            num_ref_row = x.loc[clk.index - 1]
            time_elapsed = int(clk['timestamp']) - int(num_ref_row['timestamp'])
        
            appear = 0
            for el in interacted:
                if el == clicked:
                    appear +=1
                    break
                    
            label = None
            if appear>0:
                label = 1
            else:
                label = 0
            
            return pd.DataFrame({'time_elapsed': [time_elapsed], "info_perc": [info_perc],
                                 "rating_perc": [rating_perc], "deals_perc": [deals_perc], "image_perc": [image_perc], "clickout_count": [clickout_count],
                                 "poi_count": [poi_count], "search_destination_count": [search_destination_count], "search_item_count": [search_item_count], 
                                 "filter_sel_count": [filter_sel_count], "sort_change_count": [sort_change_count],  
                                 'last_time_from_first_interaction': [last_time_from_first_interaction], 'prev_clickout': [prev_clickout], 'label': [label]})

In [481]:
dataset = df[:3000000].groupby(['user_id', 'session_id']).progress_apply(func)

HBox(children=(IntProgress(value=0, max=172357), HTML(value='')))

In [482]:
dataset.tail(200)

,,,clickout_count,deals_perc,filter_sel_count,image_perc,info_perc,label,last_time_from_first_interaction,poi_count,prev_clickout,rating_perc,search_destination_count,search_item_count,sort_change_count,time_elapsed
user_id,session_id,,,,,,,,,,,,,,,
ZWT8L0YRZJNA,e84ad7c3110b8,0,0,0.000000,0,0.000000,0.000000,1,31,0,0,0.000000,1,1,0,31
ZWT9193C78K6,b1ee2af885f70,0,0,0.000000,0,1.000000,0.000000,1,18,0,0,0.000000,0,0,0,18
ZWTWTW5O4EZQ,1ebd36b4caf45,0,1,0.000000,1,0.800000,0.000000,1,127,0,0,0.000000,0,0,2,13
ZWWD7JL6SVNR,2ada77d3fb645,0,0,0.000000,0,1.000000,0.000000,1,1,0,0,0.000000,0,0,0,1
ZWWKY70DEGX4,3599aa903d5e3,0,1,0.000000,0,0.000000,0.000000,1,215,0,1,0.000000,0,0,0,215
ZWX6MQDGJDB6,05e847f55f92a,0,2,0.000000,0,0.000000,0.250000,1,105,0,0,0.250000,0,0,0,1
ZWXYY22UYE6E,6fb1270824514,0,1,0.000000,0,0.000000,0.500000,1,5,0,1,0.000000,2,0,0,3
ZWYN43XYK4XR,e3e029fdc6bf6,0,0,0.000000,0,1.000000,0.000000,1,2076,0,0,0.000000,1,0,0,9
ZWYZJEHWK7PJ,96ffc821bf416,0,3,0.000000,4,0.000000,0.000000,0,559,0,0,0.000000,1,0,0,30


In [483]:
print(len(dataset[ dataset["label"] == 0]))
    

33898


In [484]:
X, y = dataset.iloc[:,list(range(0,5)) + list(range(6,12))],dataset.iloc[:,5]

In [485]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [486]:
import xgboost as xgb

In [536]:
xg_reg = xgb.XGBRegressor(objective ='reg:logistic')#, colsample_bytree = 0.32, learning_rate = 0.95,
                #max_depth = 1, alpha = 1, n_estimators = 10)

In [537]:
xg_reg.fit(X_train,y_train)

preds_p = xg_reg.predict(X_test)

In [538]:
preds_p

array([0.79143935, 0.67553896, 0.9983286 , ..., 0.86043465, 0.9222116 ,
       0.15501356], dtype=float32)

In [562]:
 preds = [1 if x>0.37 else 0 for x in preds_p]

In [563]:
target_names = ['clk_diff_ref', 'clk_eq_ref']
print(classification_report(y_test, preds, target_names=target_names))

              precision    recall  f1-score   support

clk_diff_ref       0.90      0.55      0.68      6758
  clk_eq_ref       0.79      0.97      0.87     11765

 avg / total       0.83      0.81      0.80     18523



In [564]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [565]:
accuracy = accuracy_score(y_test, [0]*len(preds))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 36.48%


In [566]:
accuracy = accuracy_score(y_test, [1]*len(preds))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.52%


In [567]:
accuracy = accuracy_score(y_test,  preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.35%


In [568]:
len([x for x in preds if x==0])

4096